<h3> Importações iniciais


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

In [8]:
df_test = pd.read_csv("/content/test.csv")
df_train = pd.read_csv("/content/train.csv")


<h3> Análise exploratória

In [9]:
print(df_train.shape,df_test.shape)

(17836, 12) (7644, 11)


In [10]:
df_test.head()

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral
0,EZYV22339,Ásia,Ensino Superior,S,N,2414,1973,Nordeste,103320.80,Ano,N
1,EZYV9333,Ásia,Mestrado,N,S,2928,1937,Oeste,49786.35,Ano,S
2,EZYV9740,Ásia,Ensino Superior,S,N,1836,1999,Oeste,60855.83,Ano,S
3,EZYV3328,Ásia,Ensino Superior,S,N,122,2004,Sul,82379.06,Semana,S
4,EZYV12321,América do Sul,Ensino Superior,S,S,1720,2003,Nordeste,6719.81,Ano,S


In [11]:
df_train.describe() #Aqui notamos que tem uma red flag com o salário, está muito discrepante o min e o max.

,num_de_empregados,ano_de_estabelecimento,salario_prevalecente
count,17836.00000,17836.000000,17836.000000
mean,5726.50185,1979.494898,74327.428401
std,23321.37739,42.097194,52795.859573
min,-26.00000,1800.000000,2.136700
25%,1023.00000,1976.000000,33892.912500
50%,2117.00000,1997.000000,70106.920000
75%,3504.25000,2005.000000,107564.715000
max,602069.00000,2016.000000,319210.270000


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17836 entries, 0 to 17835
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id_do_caso                      17836 non-null  object 
 1   continente                      17836 non-null  object 
 2   educacao_do_empregado           17836 non-null  object 
 3   tem_experiencia_de_trabalho     17836 non-null  object 
 4   requer_treinamento_de_trabalho  17836 non-null  object 
 5   num_de_empregados               17836 non-null  int64  
 6   ano_de_estabelecimento          17836 non-null  int64  
 7   regiao_de_emprego               17836 non-null  object 
 8   salario_prevalecente            17836 non-null  float64
 9   unidade_de_salario              17836 non-null  object 
 10  posicao_em_tempo_integral       17836 non-null  object 
 11  status_do_caso                  17836 non-null  object 
dtypes: float64(1), int64(2), object(

Descobrindo as variáveis categóricas

In [13]:
lista_var_categoricas = [x for x in df_train.columns if df_train[x].dtypes == 'object']
df_train[lista_var_categoricas]


,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,regiao_de_emprego,unidade_de_salario,posicao_em_tempo_integral,status_do_caso
0,EZYV10567,Europa,Ensino Médio,N,S,Sul,Ano,S,Negado
1,EZYV5505,Ásia,Mestrado,S,N,Meio-Oeste,Ano,S,Aprovado
2,EZYV5207,Ásia,Ensino Médio,N,N,Ilha,Ano,S,Negado
3,EZYV7545,Ásia,Ensino Médio,N,N,Meio-Oeste,Semana,S,Negado
4,EZYV16071,Ásia,Ensino Superior,S,N,Nordeste,Ano,S,Aprovado
...,...,...,...,...,...,...,...,...,...
17831,EZYV17821,Ásia,Mestrado,S,N,Oeste,Ano,S,Aprovado
17832,EZYV13200,Europa,Ensino Médio,N,N,Sul,Ano,S,Negado
17833,EZYV8903,América do Norte,Mestrado,N,N,Nordeste,Ano,S,Negado
17834,EZYV6163,América do Norte,Doutorado,S,N,Meio-Oeste,Ano,S,Aprovado


In [14]:
listas_unicas = [df_train[x].unique() for x in lista_var_categoricas] #vendo as variáveis de cada coluna

<h3>Lidando com variaveis categóricas




<h4> ordinal encoding

In [15]:
lista_var_categoricas.remove('id_do_caso')

In [16]:
lista_var_categoricas

['continente',
 'educacao_do_empregado',
 'tem_experiencia_de_trabalho',
 'requer_treinamento_de_trabalho',
 'regiao_de_emprego',
 'unidade_de_salario',
 'posicao_em_tempo_integral',
 'status_do_caso']

In [17]:
lista_var_categoricas_test = lista_var_categoricas[:]
lista_var_categoricas_test.remove("status_do_caso")

In [18]:
encoder = OrdinalEncoder(categories=[
    ['Europa', 'Ásia', 'América do Sul', 'América do Norte', 'África', 'Oceania'],
    ['Ensino Médio', 'Mestrado', 'Ensino Superior', 'Doutorado'],
    ['N', 'S'],
    ['N', 'S'],
    ['Sul', 'Meio-Oeste', 'Ilha', 'Nordeste', 'Oeste'],
    ['Ano', 'Semana', 'Hora', 'Mês'],
    ['N', 'S'],
    ['Negado', 'Aprovado']


])
encoder_test= OrdinalEncoder(categories=[
    ['Europa', 'Ásia', 'América do Sul', 'América do Norte', 'África', 'Oceania'],
    ['Ensino Médio', 'Mestrado', 'Ensino Superior', 'Doutorado'],
    ['N', 'S'],
    ['N', 'S'],
    ['Sul', 'Meio-Oeste', 'Ilha', 'Nordeste', 'Oeste'],
    ['Ano', 'Semana', 'Hora', 'Mês'],
    ['N', 'S']]
    )


In [19]:
df_train[lista_var_categoricas] = encoder.fit_transform(df_train[lista_var_categoricas])
df_test[lista_var_categoricas_test] = encoder_test.fit_transform(df_test[lista_var_categoricas_test])


In [20]:
df_test.sample(5) #confirmando se o encoder foi feito

,id_do_caso,continente,educacao_do_empregado,tem_experiencia_de_trabalho,requer_treinamento_de_trabalho,num_de_empregados,ano_de_estabelecimento,regiao_de_emprego,salario_prevalecente,unidade_de_salario,posicao_em_tempo_integral
2432,EZYV8624,2.0,2.0,0.0,0.0,2872,2003,0.0,105011.12,0.0,1.0
5074,EZYV3555,1.0,2.0,0.0,0.0,2545,1981,3.0,73305.93,0.0,1.0
473,EZYV7831,1.0,3.0,1.0,0.0,1361,1995,3.0,69343.27,0.0,1.0
5464,EZYV13855,0.0,3.0,1.0,1.0,2562,1884,4.0,47976.59,0.0,1.0
601,EZYV9161,3.0,1.0,1.0,0.0,1967,2011,3.0,58008.85,0.0,1.0


In [21]:
id_train_guardar = df_train["id_do_caso"]

y_train = df_train[["status_do_caso"]]
X_train = df_train.drop(["status_do_caso", "id_do_caso"], axis=1)

X_test = df_test.drop(["id_do_caso"], axis=1)
y_train.sample(5)


,status_do_caso
6733,0.0
11410,0.0
572,1.0
4329,1.0
15312,1.0


In [22]:
forest_model = RandomForestRegressor()
forest_model.fit(X_train, y_train)
predict = forest_model.predict(X_test)


<ipython-input-22-f41c90e4da0a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest_model.fit(X_train, y_train)


In [23]:
predict

array([0.84, 0.66, 0.67, ..., 1.  , 1.  , 0.77])

In [24]:
def transforma_predict(predicao):
  posicao = 0 
  predicao = (predicao)

  for x in range(len(predicao)):

    if predicao[x] > 0.70:

      predicao[posicao] = 1.0

    else:
      predicao[posicao] = 0.0

    predicao += 1

  return predicao

predicoes = transforma_predict(predict)
predicoes

array([2.00000e+00, 7.64466e+03, 7.64467e+03, ..., 7.64500e+03,
       7.64500e+03, 7.64477e+03])

In [25]:
#df_test2.to_csv("submissao_dummie2.csv", index=False)
